# Integrating all Modules

<center>

<figure>
    <img src="./img/vit.gif" alt="Transformer Encoder" width="50%" height="50%">
</figure>
<figcaption style="text-align:center; font-size:15px; color:#808080; margin-top:40px">
    "Image from: https://github.com/lucidrains/vit-pytorch"
  </figcaption>

</center>

마무리 하는 김에, 좋은 이미지가 있어서 포함 시켰습니다. 

이번 장에서는, 이전 장의 내용과 더불어 MLP ( Multi-Layer-Perceptron ) 를 추가하여 ViT 를 완성하도록 하겠습니다. 

이미지상에서 특이한 부분은 MLP-Head 가 상당히 왼쪽에 치우쳐져 있다는 점 입니다. 

이는, Transformer Encoder 의 출력값 중에 0번째 patch 에 해당하는,\
즉, 순서 상으로 label patch 에 해당하는 출력값을 사용한다는 것입니다.

논문에서 설명하는 방법으로는 label patch 위치의 출력만 사용하지만, \
많은 구현 내용을 참고한 결과, 전체 patch 의 mean 으로 사용하는 경우가 많이 있었습니다.

이 부분을 그대로 녹여서 ViT 를 구현해 보도록 하죠.

In [2]:
import torch
import torch.nn.functional as F
import matplotlib.pyplot as plt

import numpy as np
from torch import nn
from torch import Tensor
from PIL import Image
from torchvision.transforms import Compose, Resize, ToTensor
from einops import rearrange, reduce, repeat
from einops.layers.torch import Rearrange, Reduce
from torchsummary import summary
from collections import OrderedDict
from typing import Optional

from utils.vit_utils import Image_Embedding # 이전 장의 image embedding
from utils.vit_utils import TransformerEncoder # 이전 장의 TransformerEncoder

device = 'cuda' if torch.cuda.is_available() else 'cpu'

In [3]:
ims = torch.Tensor(np.load('./resources/test_images.npy', allow_pickle=False))
ims = rearrange(ims, 'b h w c -> b c h w')
print(type(ims), ims.shape)

<class 'torch.Tensor'> torch.Size([6, 3, 96, 96])


In [4]:
image_embedding = Image_Embedding(image_size = ims.shape[1:], patch_size=16).to(device)
embedded_tensor = image_embedding(ims.to(device))

transformerencoder = TransformerEncoder(embedding_size = 768, num_heads = 8).to(device)
encoder_output = transformerencoder(embedded_tensor)
print('Image Embedding shape:', embedded_tensor.shape, '\nEncoder output shape:', encoder_output.shape)

Image Embedding shape: torch.Size([6, 37, 768]) 
Encoder output shape: torch.Size([6, 37, 768])


---------

##  MLP-Head 

위에서는 이전 장의 내용반복 이므로, 넘어가도록 하죠.

앞에서 언급했듯, Tensor 를 Reduce 하는 방식에 집중해 주세요.

In [5]:
class MLP_Head(nn.Module):
    def __init__(self, embedding_size: int = 768, n_classes: int = 1000, reduce_type: Optional[str] = None):
        super(MLP_Head, self).__init__()
        self.reduce_type = reduce_type
        self.r_layer = self.reducelayer()
        self.layers = nn.Sequential(
                nn.LayerNorm(embedding_size), 
                nn.Linear(embedding_size, n_classes)
                )
        
    def reducelayer(self):
        if self.reduce_type == None:
            return lambda x: x[:, 0]
        elif self.reduce_type == 'mean':
            return Reduce('b p e -> b e', reduction='mean')
    
    def forward(self, x):
        x = self.r_layer(x)
        return self.layers(x)

기본적인 `layer` 는 `layernorm` 과 `linear layer` 로 구성이 되어 있습니다. 

`linear layer` 는 embedding size 를 입력 받아, classification 하는 class 의 개수로 mapping 하게 되어 있습니다. 

`reducelayer()` 함수는 `redyce_type` 이라는 hyper-parameter 에 의해 patch 의 mean 값을 사용할 수 있고, 

논문의 구현 내용처럼 patch 의 가장 첫 번째값 만을 사용할 수도 있습니다.

einops 의 사용은 이제 익숙하실 것이라 믿습니다.

이제 MLP-Head 의 입$\cdot{}$출력 차원을 확인해 보시죠!

In [6]:
mlp_head = MLP_Head().to(device)
result = mlp_head(encoder_output)
print('Encoder output shape:', encoder_output.shape, '\nResult shape:', result.shape)

Encoder output shape: torch.Size([6, 37, 768]) 
Result shape: torch.Size([6, 1000])


------------

## ViT 

이제 앞에서 다룬 모든 module 들을 사용하여 ViT 를 구현해 보죠!

In [7]:
class ViT(nn.Module):
    def __init__(self,     
                img_size: list[int, int, int],
                patch_size: int = 16,
                embedding_size: int = 768,
                depth: int = 12,
                n_classes: int = 1000,
                reduce_type: Optional[str] = None,
                **kwargs):
        super(ViT, self).__init__()
        
        self.layers = nn.Sequential(
            Image_Embedding(img_size, patch_size, embedding_size),
            TransformerEncoder(depth, **kwargs),
            MLP_Head(n_classes=n_classes, reduce_type=reduce_type),
            torch.nn.Softmax(dim=-1)
        )
        
    def forward(self, x):
        return self.layers(x)


In [8]:
summary(ViT(img_size=ims.shape[1:]).to(device), (3, 96, 96), device=device)

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
         LayerNorm-1            [-1, 3, 96, 96]          55,296
            Conv2d-2            [-1, 768, 6, 6]         590,592
         LayerNorm-3            [-1, 768, 6, 6]          55,296
   Image_Embedding-4              [-1, 37, 768]               0
         LayerNorm-5              [-1, 37, 768]           1,536
            Linear-6              [-1, 37, 768]         590,592
            Linear-7              [-1, 37, 768]         590,592
            Linear-8              [-1, 37, 768]         590,592
            Linear-9              [-1, 37, 768]         590,592
Multi_Head_Attention-10              [-1, 37, 768]               0
          Dropout-11              [-1, 37, 768]               0
ResidualConnection-12              [-1, 37, 768]               0
        LayerNorm-13              [-1, 37, 768]           1,536
           Linear-14             [-

다음 장에서는 구현된 ViT 를 사용하여 간단한 train 과 inference 를 다뤄보도록 합시다.